In [ ]:

# 네이버 뉴스 데이터를 Chroma DB에 저장하는 파이프라인
import json
import os
from typing import List, Dict, Any
from datetime import datetime
import re

# LangChain imports
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 환경 변수 설정 (OpenAI API 키가 필요합니다)
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"  # 실제 API 키로 변경해주세요
from langfuse.langchain import CallbackHandler

# Langfuse 콜백 핸들러 생성
langfuse_handler = CallbackHandler()


print("라이브러리 로드 완료")


In [ ]:
# json 파일 읽기
import json

with open('data/naver_news.json', 'r') as f:
    data = json.load(f)
news_urls = [article['url'] for article in data['articles']]

In [ ]:
# 1. 네이버 뉴스 데이터 로드 및 전처리 함수
def load_and_extract_naver_news(json_file_path: str) -> List[Dict[str, Any]]:
    """
    네이버 뉴스 JSON 파일에서 필요한 필드만 추출
    
    Args:
        json_file_path: JSON 파일 경로
        
    Returns:
        추출된 뉴스 데이터 리스트
    """
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    extracted_articles = []
    
    for article in data.get('articles', []):
        # 필요한 필드만 추출
        extracted_article = {
            'title': article.get('title', ''),
            'content': article.get('content', ''),
            'url': article.get('url', ''),
            'published_date': article.get('published_date', ''),
            'author': article.get('author', ''),
            'source': article.get('source', ''),
            'category': article.get('category', ''),
            'crawled_at': article.get('crawled_at', ''),
        }
        
        # 빈 컨텐츠는 제외
        if extracted_article['title'] and extracted_article['content']:
            extracted_articles.append(extracted_article)
    
    print(f"총 {len(extracted_articles)}개의 뉴스 기사를 추출했습니다.")
    return extracted_articles

# 데이터 로드 테스트
json_file_path = "../data/naver_top_news_by_category_20250717_230748.json"
extracted_news = load_and_extract_naver_news(json_file_path)

# 첫 번째 기사 확인
if extracted_news:
    first_article = extracted_news[0]
    print("\n첫 번째 기사 예시:")
    print(f"제목: {first_article['title'][:100]}...")
    print(f"내용: {first_article['content'][:200]}...")
    print(f"URL: {first_article['url']}")
    print(f"발행일: {first_article['published_date']}") 
else:
    print("추출된 기사가 없습니다.")




In [ ]:
def create_documents_from_news(extracted_news: List[Dict[str, Any]]) -> List[Document]:
    documents = []
    
    for i, article in enumerate(extracted_news):
        # 뉴스 본문 문단 단위로 분할
        content = article['content']
        title = article['title']
        full_text = f"{title}\n\n{content}"
        
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            encoding_name="cl100k_base",
            separators=['\n\n', '\n', r'(?<=[.!?])\s+'],
            chunk_size=300,
            chunk_overlap=50,
            is_separator_regex=True,
            keep_separator=True,
        )
        
        chunks = text_splitter.split_text(full_text)
        
        for j, chunk in enumerate(chunks):
            if chunk.strip():
                metadata = {
                    'category': article['category'],
                    'author': article['author'],
                    'source': article['source'],
                    'title': title,
                    'url': article['url'],
                    'published_date': article['published_date'],
                    'article_id': i,
                    'chunk_id': f"{i}-{j}",  # 고유 chunk ID
                }
                
                documents.append(Document(
                    page_content=chunk.strip(),
                    metadata=metadata
                ))
    
    return documents


# 문서 생성 테스트
documents = create_documents_from_news(extracted_news)
print(f"\n총 {len(documents)}개의 문서 청크가 생성되었습니다.")

if documents:
    print(f"\n첫 번째 문서 청크 예시:")
    print(f"내용: {documents[0].page_content[:200]}...")
    print(f"메타데이터: {documents[0].metadata}")
    print(f"메타데이터: {documents[1].metadata}")  

In [ ]:
import dotenv

dotenv.load_dotenv()


In [ ]:
# 4. Chroma DB에 저장
def save_to_chroma_db(documents: List[Document], collection_name: str = "naver_news"):
    """
    문서들을 Chroma DB에 저장
    
    Args:
        documents: 저장할 Document 객체 리스트
        collection_name: 컬렉션 이름
    """
    # OpenAI Embeddings 초기화 (large 모델 사용)
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-large",
        # OpenAI API 키는 환경 변수에서 자동으로 읽어옵니다
    )
    
    # Chroma DB 저장 경로
    persist_directory = "../chroma_db_news_3"
    
    # Chroma 벡터 스토어 생성
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        collection_name=collection_name,
        persist_directory=persist_directory
    )
    
    print(f"✅ {len(documents)}개의 문서가 Chroma DB에 저장되었습니다.")
    print(f"📁 저장 경로: {persist_directory}")
    print(f"📋 컬렉션 이름: {collection_name}")
    
    return vectorstore

# 주의: 실제 실행하려면 OpenAI API 키가 필요합니다
vectorstore = save_to_chroma_db(documents)
print("Chroma DB 저장 함수가 준비되었습니다.")



In [ ]:
# test code 작성
# langchain langfuse 사용
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

chroma_db = Chroma(
    collection_name="naver_news",
    embedding_function=embeddings,
    persist_directory="chroma_db_news_3",
)
# 저장된 문서 수 확인
print("Chroma DB에 저장된 문서 수:", chroma_db._collection.count())


In [ ]:
# 검색기 지정하여 테스트 
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 5},
)

query = "금리 인하"
retrieved_docs = chroma_k_retriever.invoke(query)
print(retrieved_docs)
for doc in retrieved_docs:
    print(doc.page_content)
    import pprint
    pprint.pprint(doc)
    print("="*200)